In [1]:
from game import Game
import time
import os
import re

## Load maps

In [2]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Tutorial

In [3]:
print("This is an example of the game map:")
map = load_map(2)
game = Game(map)
game.display_map()

This is an example of the game map:
W	P1	H	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [4]:
game.get_box_locations()

[(2, 3), (3, 2), (4, 3)]

In [5]:
game.get_goal_locations()

[(1, 3), (3, 1), (5, 3)]

In [6]:
game.get_player_position()

(0, 2)

- W : Wall
- H : Human
- B : Box
- P : Portal
- G : Goal

In [7]:
for direction in ['U', 'D', 'R', 'L']:
    result = game.apply_move(direction)
    print(f"Move {direction} is valid: {result}")
    if result:
        game.display_map()

Move U is valid: False
Move D is valid: False
Move R is valid: False
Move L is valid: True
W	P1	.	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	H	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [8]:
game.apply_move('U')
game.display_map()

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	H	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [9]:
game.apply_moves(['D', 'L', 'R', 'D']) 
game.display_map()
print("Is game won?", game.is_game_won())

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	.	W	W	W
W	G2/B2	.	.	P1	W	W
W	W	W	H	W	W	W
W	W	W	G3/B3	W	W	W
W	W	W	W	W	W	W
Is game won? True


In our problem, I defined the state as a list consisting of the position of the player and the position of the boxes. The simplest definition would be the position of the player but this is not a good choice; the player might return to the same spot while the positions of the boxes are not fixed. So, I defined the state as the list of positions of the boxes.  </br> </br>
The actions are defined as the directions that the player can go in each step; U, D, L and R.  </br> </br>
Also in this problem, goal test is testing wether each box is moved their corresponding goal. 

## Solvers

To optimize the search space, we can only go through the paths that their moves are valid (Game.apply_move() returns True for them). As an example, we don't have to consider and expand a move that ends up exiting the game map. Searching all possible paths (searching ${(U | D | L | R)}^+$) is unneccesary as a lot of paths are invalid.

### BFS

BFS is a graph search algorithm that searches the search space row by row and explores all nodes at the current depth level before moving deeper. The advantages of BFS is that it is a complete and optimal algorithm. But on the other hand, its' space complexity is high ($O(b^d)$) so it is not suitable for large maps with high branching factor.

In [10]:
from collections import deque

In [11]:
def solver_bfs(game_map):
   game = Game(game_map)

   player_loc = game.get_player_position()
   box_locs = game.get_box_locations()
   path = []
   root_state = [player_loc] + box_locs + [path]
   visited = set()
   q = deque()
   q.append(root_state)

   while q:
      state = q.popleft()
      path = state[-1]

      if tuple(state[:-1]) in visited:
         continue

      else:
         visited.add(tuple(state[:-1]))

         parent_state = state
         for d in ['U', 'D', 'L', 'R']:
            game.set_player_position(state[0])
            game.set_box_positions(state[1:-1])
            res = game.apply_move(d)

            if not res:
               continue
   
            else:
               if game.is_game_won():
                  return path + [d], len(visited)
               
               else:
                  box_locs = game.get_box_locations()
                  player_loc = game.get_player_position()
                  new_state = [player_loc] + box_locs + [path[:] + [d]]
                  q.append(new_state)

                  game.set_player_position(parent_state[0])
                  game.set_box_positions(parent_state[1: -1])

   return None, 0

### DFS

DFS is another graph traversal algorithm that explores as far as possible along one branch before backtracking. It uses a stack (LIFO) (can be implemented recursively or iteratively). Its' advantage is that it is more memory efficient than BFS ($O(d)$) and it is a complete algorithm. But it does not guarantee finding the optimal solution so it is not optimal.

In [12]:
# TODO: Must return moves, number of visited states
def solver_dfs(game_map):
   game = Game(game_map)
   # game.display_map()
   
   ans_path = None
   visited = set()

   player_loc = game.get_player_position()
   box_locs = game.get_box_locations()
   path = []

   root_state = [player_loc] + box_locs + [path]

   ans_path = dfs(game, visited, root_state, ans_path)

   if ans_path is None:
      return None, 0
   else:
      return ans_path, len(visited)

def dfs(game, visited, state, ans_path):
   if tuple(state[:-1]) in visited:
      return ans_path

   visited.add(tuple(state[:-1]))

   for d in ['U', 'D', 'L', 'R']:
      game.set_player_position(state[0])
      game.set_box_positions(state[1:-1])

      move_res = game.apply_move(d)

      if move_res:
         new_box_locs = game.get_box_locations()
         new_player_loc = game.get_player_position()
         path = state[-1]
         new_state = [new_player_loc] + new_box_locs + [path + [d]]

         if game.is_game_won():
            ans_path = new_state[-1]
            return ans_path

         ans_path = dfs(game, visited, new_state, ans_path)
         game.set_player_position(state[0])
         game.set_box_positions(state[1:-1])

   return ans_path

### IDS

IDS is a combination of BFS and DFS. This algorithm iteratively runs DFS with a maximum depth which this maximum depth is increased by one each time. IDS solves the memory issue of BFS and the completeness issue of DFS. It combines the advantages of both; it uses low memory (O(d)) and it is complete and optimal (if all costs are uniform).

In [27]:
# TODO: Must return moves, number of visited states
def solver_ids(game_map):
   game = Game(game_map)
   depth_limit = 50

   player_loc = game.get_player_position()
   box_locs = game.get_box_locations()
   path = []
   root_state = [player_loc] + box_locs + [path]

   for limit in range(1, depth_limit + 1):
      visited = set()
      path, found = limited_dfs(game, visited, root_state, limit)
      if found:
         return path, len(visited)
   
   return None, 0

def limited_dfs(game, visited, state, depth_limit):

   if len(state[-1]) >= depth_limit:
    return None, False
   
   if tuple(state[:-1] + [len(state[-1])]) in visited:
      return None, False

   visited.add(tuple(state[:-1] + [len(state[-1])]))

   for d in ['U', 'D', 'L', 'R']:
      game.set_player_position(state[0])
      game.set_box_positions(state[1:-1])

      move_res = game.apply_move(d)

      if move_res:
         new_box_locs = game.get_box_locations()
         new_player_loc = game.get_player_position()
         path = state[-1]
         new_state = [new_player_loc] + new_box_locs + [path + [d]]

         if game.is_game_won():
            path = new_state[-1]
            return path, True

         path, found = limited_dfs(game, visited, new_state, depth_limit)

         if found:
            return path, True

   return None, False

### A*

A* is a graph search algorithm that finds the shortest path from a start node to a goal node using both cost and heuristics. It expands the node n where f(n) = g(n) + h(n) is the lowest at each step. Here g(n) refers to the cost of the path and h(n) is the heuristic function that estimates the distance from the goal node to the current node. The algorithm is guaranteed to find the optimal path to the goal node if we are using tree search with admissible heuristic function. </br> </br>
The heuristic function I used for this problem considers the fact that the map might have portals so the manhattan distance between the player and the box is not necessarily an admissible heuristic. For each box, the path consists of two parts; 1) The path that the player takes to reach the box and 2) The path from the box to its' corresponding goal. In each of the two parts a portal might or might not be used. I set the heuristic as the minimum manhattan distance considering wether the player does or does not use a portal. This heuristic is admissible as it is always a lower bound for the distance.

In [107]:
# TODO
def manhattan_distance(pos1, pos2):
   return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def heuristic(game):
   box_locs = game.get_box_locations()
   goal_locs = game.get_goal_locations()
   portal_locs = game.get_portal_locations()
   player_loc = game.get_player_position()

   cost_approx = 0
   for i in range(len(box_locs)):
      curr_box = box_locs[i]
      curr_goal = goal_locs[i]
      
      if (curr_box == curr_goal):
         continue

      player_dist = manhattan_distance(player_loc, curr_box)
      box_dist = manhattan_distance(curr_box, curr_goal)

      box_dist1 = 1000
      player_dist1 = 1000
      for j in range(len(portal_locs)):
         curr_portal = portal_locs[j]
         
         box_dist1 = min(manhattan_distance(curr_box, curr_portal[0]) + manhattan_distance(curr_goal, curr_portal[1]) - 1, box_dist1)
         box_dist1 = min(manhattan_distance(curr_box, curr_portal[1]) + manhattan_distance(curr_goal, curr_portal[0]) - 1, box_dist1)

         player_dist1 = min(manhattan_distance(player_loc, curr_portal[0]) + manhattan_distance(box_locs[i], curr_portal[1]) - 1, player_dist1)
         player_dist1 = min(manhattan_distance(player_loc, curr_portal[1]) + manhattan_distance(box_locs[i], curr_portal[0]) - 1, player_dist1)
         
      cost_approx += min(box_dist, box_dist1) + min(player_dist, player_dist1)
   
   return cost_approx

In [260]:
def astar(game, visited, state, w = 1):
   if tuple(state[:-1]) in visited:
      return None, False

   visited.add(tuple(state[:-1]))

   valid_moves = []
   move_costs = []

   for d in ['U', 'D', 'L', 'R']:
      game.set_player_position(state[0])
      game.set_box_positions(state[1:-1])

      if game.apply_move(d):
         valid_moves.append(d)
         cost = len(state[-1]) + w * heuristic(game)
         move_costs.append(cost)
         game.set_player_position(state[0])
         game.set_box_positions(state[1:-1])

   # game.display_map()
   # print(state[-1])
   # print(move_costs)
   # print(valid_moves)
   
   sorted_costs = sorted(enumerate(move_costs), key = lambda x: x[1])
   original_indexes = [idx for idx, val in sorted_costs]

   for idx in original_indexes:
      d = valid_moves[idx]
      game.set_player_position(state[0])
      game.set_box_positions(state[1:-1])
      game.apply_move(d)
      new_box_locs = game.get_box_locations()
      new_player_loc = game.get_player_position()
      path = state[-1]
      new_state = [new_player_loc] + new_box_locs + [path + [d]]

      if game.is_game_won():
         path = new_state[-1]
         return path, True

      path, found = astar(game, visited, new_state, w)

      if found:
         return path, True
   
   return None, False

# TODO: Must return moves, number of visited states
def solver_astar(game_map, heuristic_func=heuristic, weight=1):
   game = Game(game_map)
   # game.display_map()
   player_loc = game.get_player_position()
   box_locs = game.get_box_locations()
   path = []
   visited = set()
   root_state = [player_loc] + box_locs + [path]

   path, found = astar(game, visited, root_state, weight)
   
   if found:
      return path, len(visited)
   else:
      return None, 0

## Solve

In [261]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar
}

In [17]:
def solve(map, method):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    moves, numof_visited_states = SOLVERS[method](game_map)
    end_time = time.time()
    print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")
            

In [275]:
solve(10, "A*")

A* took 5.65 seconds on map 10 and visited 191233 states.
280 moves were used: ['U', 'U', 'L', 'D', 'D', 'D', 'R', 'R', 'U', 'U', 'U', 'R', 'R', 'R', 'R', 'D', 'R', 'U', 'L', 'L', 'D', 'L', 'U', 'L', 'U', 'U', 'R', 'D', 'L', 'D', 'D', 'R', 'U', 'R', 'D', 'D', 'R', 'U', 'R', 'D', 'D', 'L', 'U', 'R', 'U', 'L', 'D', 'L', 'U', 'L', 'D', 'R', 'U', 'R', 'R', 'R', 'R', 'D', 'D', 'L', 'R', 'U', 'L', 'D', 'L', 'L', 'D', 'R', 'R', 'D', 'L', 'L', 'L', 'U', 'U', 'U', 'R', 'U', 'R', 'D', 'R', 'D', 'L', 'U', 'L', 'L', 'D', 'D', 'R', 'U', 'R', 'U', 'L', 'D', 'U', 'L', 'D', 'L', 'U', 'L', 'U', 'R', 'U', 'R', 'D', 'D', 'D', 'L', 'U', 'R', 'R', 'R', 'R', 'D', 'D', 'L', 'D', 'D', 'D', 'R', 'U', 'U', 'D', 'D', 'L', 'U', 'U', 'U', 'R', 'U', 'L', 'L', 'L', 'D', 'D', 'L', 'L', 'U', 'R', 'R', 'U', 'R', 'U', 'L', 'L', 'D', 'L', 'U', 'R', 'D', 'L', 'U', 'U', 'L', 'D', 'R', 'D', 'D', 'R', 'R', 'U', 'D', 'L', 'U', 'U', 'R', 'U', 'R', 'D', 'D', 'U', 'U', 'L', 'D', 'D', 'D', 'R', 'U', 'L', 'D', 'D', 'D', 'D', 'R', 

In [183]:
def solve_all():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        for method in SOLVERS.keys():
            solve(map, method)
            

In [ ]:
solve_all() # Solve all maps using all methods

| Map       | No. of Visited States | Problem Solution                                                                                                   | Average Execution Time (s) |
|-----------|----------------------:|-------------------------------------------------------------------------------------------------------------------|---------------------------:|
| Map 1 - BFS  | 40  | ['U', 'D', 'D', 'U', 'L', 'R', 'R']                                                                                      | 0.0  |
| Map 2 - BFS  | 18  | ['L', 'U', 'D', 'D', 'U', 'L']                                                                                         | 0.0  |
| Map 3 - BFS  | 111 | ['U', 'L', 'D', 'D', 'U', 'U', 'U', 'U', 'R', 'D', 'D', 'D', 'D']                                                       | 0.0  |
| Map 5 - BFS  | 4770 | ['U', 'L', 'D', 'D', 'R', 'D', 'L', 'L', 'L', 'U', 'U', 'U', 'R', 'U', 'L']                                           | 0.05 |
| Map 6 - BFS  | 34  | ['U', 'U', 'U', 'U', 'U', 'R', 'R', 'R', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'R', 'D', 'L', 'R', 'R', 'R', 'R', 'R', 'R', 'R'] | 0.16 |
| Map 7 - BFS  | 34  | ['R', 'U', 'R', 'R', 'D', 'D', 'D', 'D', 'L', 'D', 'R', 'U', 'U', 'U', 'U', 'L', 'L', 'L', 'R', 'D', 'R', 'D', 'R', 'D', 'D', 'L', 'L', 'D', 'L', 'L', 'U', 'U', 'D', 'R'] | 5.52 |
| Map 8 - BFS  | 14  | ['U', 'U', 'R', 'D', 'L', 'D', 'R', 'R', 'D', 'R', 'U', 'R', 'D', 'R']                                                 | 0.06 |
| Map 10 - BFS | 228003 | ['R', 'R', 'R', 'R', 'R', 'D', 'R', 'U', 'L', 'U', 'R', 'U', 'L', 'L', 'L', 'U', 'L', 'D', 'R', 'U', 'U', 'U', 'L', 'D', 'R', 'D', 'L', 'D', 'R', 'R', 'D', 'R', 'U', 'L', 'U', 'R', 'U', 'R', 'D', 'D', 'R', 'D', 'L', 'L', 'L', 'L'] | 3.25 |
| Map 1 - DFS  | 43  | ['D', 'U', 'L', 'R', 'R', 'L', 'U']                                                                                   | 0.0  |
| Map 2 - DFS  | 20  | ['L', 'D', 'U', 'L', 'R', 'U']                                                                                        | 0.0  |
| Map 3 - DFS  | 134 | ['U', 'L', 'U', 'U', 'R', 'D', 'U', 'L', 'D', 'D', 'D', 'U', 'U', 'U', 'R', 'D', 'D', 'D', 'D', 'U', 'U', 'U', 'U', 'L', 'D', 'D', 'D', 'D'] | 0.0  |
| Map 5 - DFS  | 13769 | (Long solution path omitted for readability)                                                                       | 0.135 |
| Map 1 - IDS  | 18  | ['U', 'D', 'D', 'U', 'L', 'R', 'R']                                                                                   | 0.0  |
| Map 2 - IDS  | 8   | ['L', 'U', 'D', 'D', 'U', 'L']                                                                                       | 0.0  |
| Map 3 - IDS  | 292 | ['U', 'L', 'D', 'D', 'U', 'U', 'U', 'U', 'R', 'D', 'D', 'D', 'D']                                                    | 0.1  |
| Map 5 - IDS  | 6288 | ['U', 'L', 'D', 'D', 'R', 'D', 'L', 'L', 'L', 'U', 'U', 'U', 'R', 'U', 'L']                                         | 0.2  |
| Map 8 - IDS  | 4991 | ['U', 'U', 'R', 'D', 'L', 'D', 'R', 'R', 'D', 'R', 'U', 'R', 'D', 'R']                                             | 0.2  |

| Map                     | No. of Visited States | Problem Solution                                                       | Average Execution Time (s) |
|-------------------------|----------------------:|------------------------------------------------------------------------|---------------------------:|
| Map 1 - A*             |                     7 | ['U', 'D', 'D', 'U', 'L', 'R', 'R']                                   |                        0.0 |
| Map 2 - A*             |                     6 | ['L', 'U', 'D', 'D', 'U', 'L']                                        |                        0.0 |
| Map 3 - A*             |                    14 | ['U', 'L', 'U', 'U', 'R', 'D', 'D', 'D', 'D', 'U', 'U', 'L', 'D', 'D'] |                        0.0 |
| Map 5 - A*             |                    32 | a path of length 29                                                   |                        0.0 |
| Map 6 - A*             |                    75 | a path of length 75                                                   |                        0.0 |
| Map 7 - A*             |                   NaN | NaN                                                                    |                        NaN |
| Map 8 - A*             |                  7167 | a path of length 156                                                  |                       0.24 |
| Map 10 - A*            |                191233 | a path of length 280                                                  |                       5.65 |
| Map 1 - Weighted A*    |                   NaN | NaN                                                                    |                        NaN |
| Map 2 - Weighted A*    |                   NaN | NaN                                                                    |                        NaN |
| Map 3 - Weighted A*    |                   NaN | NaN                                                                    |                        NaN |
| Map 5 - Weighted A*    |                   NaN | NaN                                                                    |                        NaN |
| Map 6 - Weighted A*    |                   NaN | NaN                                                                    |                        NaN |
| Map 7 - Weighted A*    |                   NaN | NaN                                                                    |                        NaN |
| Map 8 - Weighted A*    |                   NaN | NaN                                                                    |                        NaN |
| Map 9 - Weighted A*    |                   NaN | NaN                                                                    |                        NaN |
| Map 10 - Weighted A*   |                   NaN | NaN                                                                    |                        NaN |